In [234]:
import csv
import pandas as pd
import numpy as np
import sklearn

from pymongo import MongoClient

ALL_COLORS = ['W', 'U', 'B', 'R', 'G']

PATH = './data/draft_data_public.MKM.PremierDraft.csv'

NO_NO = 0
NO_YES = 1
YES_YES = 2

NO = 0
YES = 1

MAX_PIP_COUNT = 40
NUM_COLORS = 5
NUM_IN_PACK = 13
NUM_PACKS = 3

In [244]:
client = MongoClient()
cards_en = client.scryfall.cards_en

with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    COLUMNS = next(draft_data)

NAMES = [col.split('pack_card_')[1] for col in columns if 'pack_card_' in col]
PICK_IDX = columns.index('pick')
DRAFTID_IDX = columns.index('draft_id')
PACKNUM_IDX = columns.index('pack_number')
PICKNUM_IDX = columns.index('pick_number')
PACK_IDX = columns.index('pack_card_A Killer Among Us')
POOL_IDX = columns.index('pool_A Killer Among Us')
NUM_CARDS = len(names)


In [236]:
def get_card(card_name):
    card = cards_en.find_one({'name': card_name})

    if card is None:
        query = {            
            'card_faces': {
                '$elemMatch': {
                    'name': card_name
                }
            },
        }

        card = cards_en.find_one(query)

    if card is None:
        error_message = 'No match for card name {}'.format(card_name)
        print(error_message)

    return(card)

def card_vector(index, row):
    return np.array([int(x) for x in row[index:index + NUM_CARDS]])

In [238]:
alsa_filepath = './data/alsa.npy'
# calculate ALSA

# last_seen = np.zeros(len(names))
# counts = np.zeros(len(names))
# with open(PATH) as csvfile:
#     draft_data = csv.reader(csvfile)
#     next(draft_data)
#     draft_id = None
#     pack_num = None
#     row_last_seen = np.zeros(len(names))
    
#     for row in draft_data:
#         if draft_id != row[DRAFTID_IDX] or pack_num != row[PACKNUM_IDX]:
#             last_seen += row_last_seen
#             counts += np.where(row_last_seen, 1, 0)
#             draft_id = row[DRAFTID_IDX]
#             pack_num = int(row[PACKNUM_IDX])
#             row_last_seen = np.zeros(NUM_CARDS)
#         pack = card_vector(PACK_IDX, row)
#         np.putmask(row_last_seen, pack, int(row[PICKNUM_IDX]))
        
# alsa = last_seen / counts
# alsa_series = pd.Series(alsa, index=NAMES)
# alsa_series = alsa_series[pd.notna(alsa)]

# with open(alsa_filepath, 'wb') as f:
#     np.save(f, alsa)

alsa = np.load(alsa_filepath)

In [239]:
len(names)

326

In [273]:
def pip_vector(card_name):
    card = get_card(card_name)
    if card is None:
        print(card_name + ' Not Found')
    mana_cost = card['mana_cost']
    return np.array([mana_cost.count(f'{{{color}}}') for color in ALL_COLORS])  \
        + 1/2 * np.array([mana_cost.count(f'/{color}') for color in ALL_COLORS])\
        + 1/2 * np.array([mana_cost.count(f'{color}/') for color in ALL_COLORS])

PIP_LOOKUP = np.array([pip_vector(name) for name in names])

def pool_pips(pool):
    return np.matmul(pool.reshape(1,NUM_CARDS), PIP_LOOKUP)[0]

def pick_number(row):
    return int(row[PICKNUM_IDX]) + NUM_IN_PACK * int(row[PACKNUM_IDX])

def pip_counts_to_matrix(pip_counts):
    matrix = np.zeros((NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
    for i in range(NUM_COLORS):
        matrix[i][0:int(pip_counts[i])] = 1
    return matrix
       


In [296]:
num_samples = 0
experiment_counts = np.zeros((NUM_PACKS * NUM_IN_PACK, NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
event_counts = np.zeros((NUM_PACKS * NUM_IN_PACK, NUM_CARDS, len([NO_NO, NO_YES, YES_YES]), NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
received_events = np.zeros((NUM_PACKS * NUM_IN_PACK, NUM_CARDS, len([NO_NO, NO_YES, YES_YES]), NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc) 

with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    next(draft_data) # discard names

    for row in draft_data:
        experiment_counts[pick_number(row)] += pool_pre_matrix
        num_samples += 1  
        
        pack = card_vector(PACK_IDX, row) 
        pack_alsa = np.where(pack, alsa, NUM_IN_PACK+1)
        pool = card_vector(POOL_IDX, row)
        pool_pip_counts_pre = pool_pips(pool)
        pool_pre_matrix = pip_counts_to_matrix(pool_pip_counts_pre)
        pool_pip_counts_post = pool_pip_counts_pre + PIP_LOOKUP[names.index(row[PICK_IDX])]
        
        pool_post_matrix = pip_counts_to_matrix(pool_pip_counts_post)
        no_yes_matrix = pool_post_matrix - pool_pre_matrix
        no_no_matrix = 1 - pool_post_matrix

        received_events[pick_number(row), min_indices[0], NO_NO] += no_no_matrix
        received_events[pick_number(row), min_indices[0], NO_YES] += no_yes_matrix
        received_events[pick_number(row), min_indices[0], YES_YES] += pool_pre_matrix
                
        min_indices = np.argpartition(pack_alsa, 2)
        top_passed_card = min_indices[1] if names[min_indices[0]] == row[columns.index('pick')] else min_indices[0]
            
        event_counts[pick_number(row), top_passed_card, NO_NO] += no_no_matrix
        event_counts[pick_number(row), top_passed_card, NO_YES] += no_yes_matrix
        event_counts[pick_number(row), top_passed_card, YES_YES] += pool_pre_matrix

        

In [299]:
experiment_counts[13]

['expansion',
 'event_type',
 'draft_id',
 'draft_time',
 'rank',
 'event_match_wins',
 'event_match_losses',
 'pack_number',
 'pick_number',
 'pick',
 'pick_maindeck_rate',
 'pick_sideboard_in_rate',
 'pack_card_A Killer Among Us',
 'pack_card_Absolving Lammasu',
 'pack_card_Aftermath Analyst',
 'pack_card_Agency Coroner',
 'pack_card_Agency Outfitter',
 'pack_card_Agrus Kos, Spirit of Justice',
 'pack_card_Airtight Alibi',
 'pack_card_Alley Assailant',
 'pack_card_Alquist Proft, Master Sleuth',
 'pack_card_Analyze the Pollen',
 "pack_card_Anzrag's Rampage",
 'pack_card_Anzrag, the Quake-Mole',
 "pack_card_Archdruid's Charm",
 "pack_card_Assassin's Trophy",
 'pack_card_Assemble the Players',
 'pack_card_Audience with Trostani',
 "pack_card_Aurelia's Vindicator",
 'pack_card_Aurelia, the Law Above',
 'pack_card_Auspicious Arrival',
 'pack_card_Axebane Ferox',
 'pack_card_Barbed Servitor',
 'pack_card_Basilica Stalker',
 'pack_card_Behind the Mask',
 'pack_card_Benthic Criminologists',


In [ ]:
with open(PATH) as csvfile:
    rows = []
    draft_data = csv.reader(csvfile)
    while len(rows) < NUM_PACKS * NUM_IN_PACK:
        row = next(draft_data)
        if row[columns.index('draft_time')] > '2024-03-20':
            rows.append(row)



In [294]:
idx = names.index("Lightning Helix")
pick_num = 0 # pack 2 pick 1
event_counts[pick_num, idx, NO_YES]# / experiment_counts[pick_num, idx, NO]


array([[899, 140,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [535, 114,   2,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [484, 102,  28,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [679, 104,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [563,  91,   6,   0,   0,   0,   0,   0,   0,   0,   0,   0, 

In [288]:
def update_sender(sender_parameters, prior, pack_received, pack_num, pick_num):
    pack = card_vector(PACK_IDX, row) 
    pack_alsa = np.where(pack, alsa, NUM_IN_PACK+1)
    min_card = pack[np.argmin(pack_alsa)]

    yes_yes = sender_parameters[3*pack_num + pick_num, min_card, YES_YES]
    no_yes = sender_parameters[3*pack_num + pick_num, min_card, NO_YES]
    no_no = sender_parameters[3*pack_num + pick_num, min_card, NO_NO]

    # Bayesian updating for part of both sides that becomes "yes" to each distribution function
    yes_update = prior * yes_yes + (1-prior) * no_yes

    # Bayesian updating for the part that remains no
    no_update = (1 - prior) * no_no

    return yes_update / (yes_update + no_update)

def update_receiver(receiver_parameters, prior, pack_received, pack_num, pick_num,):
    pack = card_vector(PACK_IDX, row)
    pack_also = np.where(pack, alsa, NUM_IN_PACK + 1)
    
    min_indices = np.argpartition(pack_alsa, 2)
    top_passed_card = min_indices[1] if names[min_indices[0]] == row[columns.index('pick')] else min_indices[0]

    yes_yes = receiver_parameters[3*pack_num + pick_num, top_passed_card, YES_YES]
    no_yes = receiver_parameters[3*pack_num + pick_num, top_passed_card, NO_YES]
    no_no = receiver_parameters[3*pack_num + pick_num, top_passed_card, NO_NO]
    
    # Bayesian updating for part of both sides that becomes "yes" to each distribution function
    yes_update = prior * yes_yes + (1-prior) * no_yes

    # Bayesian updating for the part that remains no
    no_update = (1 - prior) * no_no

    return yes_update / (yes_update + no_update)
    
    

0